<a href="https://colab.research.google.com/github/junwoochoi33/LLMPractice/blob/main/chapter_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**1. 토큰화 코드**

In [3]:
# 띄어쓰기 단위로 분리
input_text = "나는 최근 파리 여행을 다녀왔다"
input_text_list = input_text.split()
print("input_text_list: ", input_text_list)

# 토큰 -> 아이디 딕셔너리와 아이디 -> 토큰 딕셔너리 만들기
str2idx = {word:idx for idx, word in enumerate(input_text_list)}
idx2str = {idx:word for idx, word in enumerate(input_text_list)}
print("str2idx: ", str2idx)
print("idx2str: ", idx2str)

# 토큰을 토큰 아이디로 변환
input_ids = [str2idx[word] for word in input_text_list]
print("input_ids: ", input_ids)

input_text_list:  ['나는', '최근', '파리', '여행을', '다녀왔다']
str2idx:  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
idx2str:  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}
input_ids:  [0, 1, 2, 3, 4]


## **2. 토큰 아이디에서 벡터로 변환**

In [5]:
import torch
import torch.nn as nn

embedding_dim = 16
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

input_embeddings = embed_layer(torch.tensor(input_ids)) # (5, 16)
input_embeddings = input_embeddings.unsqueeze(0) # (1, 5, 16)
# input_embeddings.shape
input_embeddings

tensor([[[-0.5010, -0.8169,  0.0062,  0.3444,  0.5279, -0.3274, -1.4388,
           0.4081, -0.0491, -0.2718,  0.8183,  1.0439,  1.2350, -1.2265,
           0.7630, -0.1012],
         [-2.5741,  1.4213,  0.4881, -0.7989, -0.4776,  2.6087,  0.1322,
           1.7640,  0.2387, -1.3405,  0.8953,  1.1727,  0.9397,  1.1423,
          -1.0968,  0.5021],
         [-1.0393,  0.0510, -0.5647, -1.9329,  1.4266,  0.4903,  0.8350,
           0.6742, -0.2432, -0.7441,  1.8056, -0.8244,  0.2992, -0.0529,
          -1.2714, -1.2380],
         [ 1.2332, -0.1728, -0.9220, -0.4600,  0.7893,  1.7829, -0.6328,
           0.6141,  0.3531, -1.6986, -1.0748, -0.4737, -1.0477,  0.6228,
          -1.3619, -0.4113],
         [ 0.1481,  0.8429,  0.1843, -0.9478, -0.8120, -0.8513, -1.1132,
          -0.8770, -0.6190,  0.7718, -1.5073, -2.0743,  1.1972, -0.7664,
           1.5601,  0.4136]]], grad_fn=<UnsqueezeBackward0>)

## **3. 절대적 위치 인코딩**

In [6]:
embedding_dim = 16 # (예: "사과" → [0.1, -0.3, ..., 0.5])
max_position = 12 # 문장은 최대 12단어까지 있다고 생각

# 토큰 임베딩 층 생성
embed_layer = nn.Embedding(len(str2idx), embedding_dim) # 단어들을 숫자 벡터로 바꿔주는 사전 (예: "사과"는 [0.1, 0.2, ..., 0.9])
# 위치 인코딩 층 생성
position_embed_layer = nn.Embedding(max_position, embedding_dim) # 단어가 몇 번째에 있는지, 그 위치도 벡터로 변경 (예: 첫 번째 단어는 [0.05, 0.02, ..., 0.1])

position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0) # (예: 단어가 5개라면 → 위치 번호는 [0, 1, 2, 3, 4])
position_encodings = position_embed_layer(position_ids) # 위치 번호를 숫자 벡터로 변경
token_embeddings = embed_layer(torch.tensor(input_ids)) # 진짜 단어들을 숫자 벡터로
token_embeddings = token_embeddings.unsqueeze(0) # 묶어서 하나의 문장(배치)로
# 토큰 임베딩과 위치 인코딩을 더해 최종 입력 임베딩 생성
input_embeddings = token_embeddings + position_encodings # 단어 정보 + 위치 정보
input_embeddings.shape # 문장 1개, 단어 5개, 숫자 16개짜리 벡터

torch.Size([1, 5, 16])